# Problema de asignación **3-dimensional** - Ruymán García Martín

In [3]:
import Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")
using JuMP, GLPK, Random, Test

  Installing known registries into `C:\Users\Ruymán\.julia`
       Added `General` registry to C:\Users\Ruymán\.julia\registries
    Updating registry at `C:\Users\Ruymán\.julia\registries\General.toml`
   Resolving package versions...
   Installed DiffRules ──────────── v1.15.1
   Installed JSON ───────────────── v0.21.4
   Installed IrrationalConstants ── v0.2.2
   Installed DiffResults ────────── v1.1.0
   Installed SpecialFunctions ───── v2.5.0
   Installed Preferences ────────── v1.4.3
   Installed BenchmarkTools ─────── v1.5.0
   Installed MutableArithmetics ─── v1.6.0
   Installed Bzip2_jll ──────────── v1.0.8+2
   Installed Parsers ────────────── v2.8.1
   Installed CodecBzip2 ─────────── v0.8.4
   Installed JLLWrappers ────────── v1.6.1
   Installed NaNMath ────────────── v1.0.2
   Installed StaticArraysCore ───── v1.4.3
   Installed OrderedCollections ─── v1.7.0
   Installed PrecompileTools ────── v1.2.1
   Installed TranscodingStreams ─── v0.11.3
   Installed ForwardDiff ───

# Problemas de asignación **3-dimensional**

In [4]:
using GLPK

## Problema de asignación 3-dimensional - Axial


Consideremos ahora $n$ videojuegos, $n$ géneros, y $n$ idiomas. Asumamos conocer el coste $c_{ijk}$ de colocar un videojuego $i$ en el género $j$ y en el idioma $k$. Lo que se busca es que cada videojuego pertenezca a un único genero a un único idioma. Igualmente queremos que cada género este en un videojuego y disponible en algún idioma, y que cada idioma este asignada a un videojuego que se encuadra en un género. Se busca la asignación con menor coste total.

In [5]:
N = 4 # número de videojuegos = géneros = idioma
Random.seed!(6)
cost = rand( 0:100, N, N, N )

4×4×4 Array{Int64, 3}:
[:, :, 1] =
 84  10   2   6
  8  17  38  46
 56  97  74  81
  1  38  26  99

[:, :, 2] =
 80  85  39  71
 14  40  84  21
 42  63  28  12
 30  12  81  39

[:, :, 3] =
 23  95  66  37
 32  42  69  81
 65  95  18  25
  5  30  87  85

[:, :, 4] =
  21  64   9  79
 100  97  87  52
   0  46  51  51
  84   9  53  37

In [6]:
model = Model(GLPK.Optimizer)
set_silent(model)

@variable(model, x[1:N,1:N,1:N] , Bin)
@objective(model, Min, sum( cost .* x ))
# En este caso las restricciones indican que cada videojuego solo
# puede pertenecer a un único género y a un solo idioma
@constraint(model, videogame[v=1:N] , sum(x[v,:,:]) == 1)
@constraint(model, genre[g=1:N] , sum(x[:,g,:]) == 1)
@constraint(model, language[l=1:N] , sum(x[:,:,l]) == 1)
println(model);

Min 84 x[1,1,1] + 8 x[2,1,1] + 56 x[3,1,1] + x[4,1,1] + 10 x[1,2,1] + 17 x[2,2,1] + 97 x[3,2,1] + 38 x[4,2,1] + 2 x[1,3,1] + 38 x[2,3,1] + 74 x[3,3,1] + 26 x[4,3,1] + 6 x[1,4,1] + 46 x[2,4,1] + 81 x[3,4,1] + 99 x[4,4,1] + 80 x[1,1,2] + 14 x[2,1,2] + 42 x[3,1,2] + 30 x[4,1,2] + 85 x[1,2,2] + 40 x[2,2,2] + 63 x[3,2,2] + 12 x[4,2,2] + 39 x[1,3,2] + 84 x[2,3,2] + 28 x[3,3,2] + 81 x[4,3,2] + 71 x[1,4,2] + 21 x[2,4,2] + [[...3 terms omitted...]] + 32 x[2,1,3] + 65 x[3,1,3] + 5 x[4,1,3] + 95 x[1,2,3] + 42 x[2,2,3] + 95 x[3,2,3] + 30 x[4,2,3] + 66 x[1,3,3] + 69 x[2,3,3] + 18 x[3,3,3] + 87 x[4,3,3] + 37 x[1,4,3] + 81 x[2,4,3] + 25 x[3,4,3] + 85 x[4,4,3] + 21 x[1,1,4] + 100 x[2,1,4] + 84 x[4,1,4] + 64 x[1,2,4] + 97 x[2,2,4] + 46 x[3,2,4] + 9 x[4,2,4] + 9 x[1,3,4] + 87 x[2,3,4] + 51 x[3,3,4] + 53 x[4,3,4] + 79 x[1,4,4] + 52 x[2,4,4] + 51 x[3,4,4] + 37 x[4,4,4]
Subject to
 videogame[1] : x[1,1,1] + x[1,2,1] + x[1,3,1] + x[1,4,1] + x[1,1,2] + x[1,2,2] + x[1,3,2] + x[1,4,2] + x[1,1,3] + x[1,2,3] + x

In [7]:
status = optimize!(model)
@show termination_status(model)
println("Total: ", objective_value(model))
println("Decision variables: ", value.(x))
for v=1:N, g=1:N, l=1:N
    if value(x[v,g,l]) > 0.5
        println("El videojuego $v se encuadra en el género $g con el idioma $l")
    end
end

termination_status(model) = MathOptInterface.OPTIMAL
Total: 43.0
Decision variables: [0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0;;; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
El videojuego 1 se encuadra en el género 3 con el idioma 4
El videojuego 2 se encuadra en el género 2 con el idioma 1
El videojuego 3 se encuadra en el género 4 con el idioma 2
El videojuego 4 se encuadra en el género 1 con el idioma 3


## Problema de asignación 3-dimensional - Planar

Consideremos ahora $n$ videojuegos, $n$ géneros, y $n$ idiomas. Asumamos conocer el coste $c_{ijk}$ de colocar un videojuego $i$ en el género $j$ y en el idioma $k$. Lo que se busca es que cada videojuego pertenezca a todos los géneros y a estar disponible en todos los idiomas. Igualmente queremos que cada género este en un videojuego y disponible en todos los idiomas, y que cada idioma este asignado a un videojuego que se encuadre en todos los géneros. Se busca la asignación con menor coste total.

In [8]:
N = 4 # número de videojuegos = géneros = idioma
Random.seed!(45)
cost = rand( 0:100, N, N, N )

4×4×4 Array{Int64, 3}:
[:, :, 1] =
 17  99   6  88
  3  77  55  69
 90  77  57   7
  5  63  59  15

[:, :, 2] =
 36  18  41  16
 54  80  73  62
 56  96  77  29
 71  44  72  61

[:, :, 3] =
 82  31  73  26
 32  51  91  35
  9   2  24  87
 73  93  99  39

[:, :, 4] =
 97  80  37   9
 22  61  83  30
 28  52  53   4
 38  66  94  59

In [9]:
model = Model(GLPK.Optimizer)
@variable(model, x[1:N,1:N,1:N], Bin )
@objective(model, Min, sum( cost .* x ))
# En este caso las restricciones indican que cada videojuego puede pertenecer
# a todos los géneros y a estar disponible en todos los idiomas
@constraint(model, videogame[v=1:N,g=1:N] , sum(x[v,g,:]) == 1 )
@constraint(model, genre[v=1:N,l=1:N] , sum(x[v,:,l]) == 1 )
@constraint(model, language[g=1:N,l=1:N] , sum(x[:,g,l]) == 1 )
println(model)

Min 17 x[1,1,1] + 3 x[2,1,1] + 90 x[3,1,1] + 5 x[4,1,1] + 99 x[1,2,1] + 77 x[2,2,1] + 77 x[3,2,1] + 63 x[4,2,1] + 6 x[1,3,1] + 55 x[2,3,1] + 57 x[3,3,1] + 59 x[4,3,1] + 88 x[1,4,1] + 69 x[2,4,1] + 7 x[3,4,1] + 15 x[4,4,1] + 36 x[1,1,2] + 54 x[2,1,2] + 56 x[3,1,2] + 71 x[4,1,2] + 18 x[1,2,2] + 80 x[2,2,2] + 96 x[3,2,2] + 44 x[4,2,2] + 41 x[1,3,2] + 73 x[2,3,2] + 77 x[3,3,2] + 72 x[4,3,2] + 16 x[1,4,2] + 62 x[2,4,2] + [[...4 terms omitted...]] + 9 x[3,1,3] + 73 x[4,1,3] + 31 x[1,2,3] + 51 x[2,2,3] + 2 x[3,2,3] + 93 x[4,2,3] + 73 x[1,3,3] + 91 x[2,3,3] + 24 x[3,3,3] + 99 x[4,3,3] + 26 x[1,4,3] + 35 x[2,4,3] + 87 x[3,4,3] + 39 x[4,4,3] + 97 x[1,1,4] + 22 x[2,1,4] + 28 x[3,1,4] + 38 x[4,1,4] + 80 x[1,2,4] + 61 x[2,2,4] + 52 x[3,2,4] + 66 x[4,2,4] + 37 x[1,3,4] + 83 x[2,3,4] + 53 x[3,3,4] + 94 x[4,3,4] + 9 x[1,4,4] + 30 x[2,4,4] + 4 x[3,4,4] + 59 x[4,4,4]
Subject to
 videogame[1,1] : x[1,1,1] + x[1,1,2] + x[1,1,3] + x[1,1,4] == 1
 videogame[2,1] : x[2,1,1] + x[2,1,2] + x[2,1,3] + x[2,1,4] ==

In [10]:
status = optimize!(model)
@show termination_status(model)
println("Total: ",objective_value(model))
println("Decision variables: ",value.(x))
for v=1:N, g=1:N, l=1:N
    if value(x[v,g,l])>0.5
        println("El videojuego $v se encuadra en el género $g con el idioma $l")
    end
end

termination_status(model) = MathOptInterface.OPTIMAL
Total: 618.0
Decision variables: [0.0 0.0 1.0 0.0; 1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 1.0;;; 1.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0; 0.0 1.0 0.0 0.0;;; 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 1.0; 1.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0;;; 0.0 0.0 0.0 1.0; 0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 1.0 0.0 0.0 0.0]
El videojuego 1 se encuadra en el género 1 con el idioma 2
El videojuego 1 se encuadra en el género 2 con el idioma 3
El videojuego 1 se encuadra en el género 3 con el idioma 1
El videojuego 1 se encuadra en el género 4 con el idioma 4
El videojuego 2 se encuadra en el género 1 con el idioma 1
El videojuego 2 se encuadra en el género 2 con el idioma 4
El videojuego 2 se encuadra en el género 3 con el idioma 2
El videojuego 2 se encuadra en el género 4 con el idioma 3
El videojuego 3 se encuadra en el género 1 con el idioma 3
El videojuego 3 se encuadra en el género 2 con el idioma 1
El videojuego 3 se encuadra en el género 3 co

Al ser planar, como resultado podemos ver que un videojuego puede pertenecer a todos los géneros y estar disponible en todos los idiomas.